### Consecutive Events

For each user_id, find the difference between the last action and the second last action. Action here is defined as visiting a page. If the user has just one action, you can either remove her from the final results or keep that user_id and have NULL as time difference between the two actions.

Our table shows for each user all the pages she visited and the corresponding timestamp.



In [1]:
import pandas as pd
url = 'https://raw.githubusercontent.com/data-eng-10-21/interview-questions/main/ds-sql/2-consec-events/query_users.csv'
df = pd.read_csv(url, index_col = 0)

In [2]:
import sqlite3

conn = sqlite3.connect('events.db')

In [3]:
df.to_sql('events', conn, if_exists = 'replace')

In [4]:
pd.read_sql('select * from events limit 2', conn)

,user_id,page,unix_timestamp
0,6684,home_page,1451640067
1,8098,home_page,1451647141


In [42]:
results = pd.read_sql("""
SELECT * FROM 
    (SELECT user_id, unix_timestamp, LAG(unix_timestamp, 1) OVER
    (PARTITION BY user_id order by unix_timestamp DESC) as earlier, 
    (LAG(unix_timestamp, 1) OVER
    (PARTITION BY user_id order by unix_timestamp DESC) - unix_timestamp) as delta, 
    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY unix_timestamp DESC) AS order_desc 
    FROM events)
    ;
""", conn)
results[:10]

,user_id,unix_timestamp,earlier,delta,order_desc
0,2,1451606400,NaN,NaN,1
1,3,1451606404,NaN,NaN,1
2,3,1451606402,1.451606e+09,2.0,2
3,4,1451606423,NaN,NaN,1
4,4,1451606419,1.451606e+09,4.0,2
5,4,1451606418,1.451606e+09,1.0,3
6,4,1451606407,1.451606e+09,11.0,4
7,5,1451606425,NaN,NaN,1
8,7,1451606427,NaN,NaN,1
9,8,1451606437,NaN,NaN,1


<img src="./data-desc.png" width="60%">

### Answer

<img src="./answer.png" width="40%">

```sql
SELECT user_id,  
       unix_timestamp - previous_time AS Delta_SecondLast0ne_LastOne  
FROM  
     (SELECT user_id,   
             unix_timestamp,  
             LAG(unix_timestamp, 1) OVER (PARTITION BY user_id ORDER BY unix_timestamp) AS previous_time,  
             ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY unix_timestamp DESC) AS order_desc  
       FROM query_one  
       ) tmp  
WHERE order_desc = 1  
ORDER BY user_id  
LIMIT 5;
```